# **Clase 22-4-2021** 

# Pandas 2020 2C 1

## Enunciado

Una importante compañía de e-commerce acumula información de las interacciones de sus usuarios con su plataforma mediante un motor de analytics. Esta información está disponible en un data frame para que sea analizada por nuestro equipo.


El data frame llamado `events.csv` posee las siguientes columnas:


* uid: identificador único que representa a un usuario que visita la plataforma. 

* sessionid: identificador único de la sesión que generó el usuario.

* event: nombre del evento que se generó. 

* value: información adicional que se guarda con el evento.

* datetime: timedate64 que contiene fecha y hora del evento. 

Para poder realizar un análisis es necesario tener las siguientes consideraciones:


Cada vez que un usuario visita la plataforma, el sistema le genera una nueva sesión con un `sessionid` asociado. Esto implica que un usuario puede realizar múltiples sesiones.


Los eventos son interacciones que realiza el usuario con el sistema. Una sesión está compuesta por al menos un evento.


Se le pide a nuestro equipo las siguientes tareas utilizando pandas:

a) Listar aquellos usuarios que tengan un total de sesiones mayor al promedio de sesiones por usuario del sistema. Ej: si el promedio de los usuarios cuenta con 20 sesiones, se debe listar los usuarios que tengan más de 20 sesiones asociadas. 

b) Calcular el valor promedio de los eventos 'ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.conversion' para aquellos usuarios que hayan realizado más de 25 (tomaremos 3 para el ejercicio) sesiones. Los resultados deben ser presentados en un dataframe con las siguientes columnas ('uid','ecommerce_view_product_mean', 'ecommerce_checkout_mean', 'ecommerce_conversion_mean'). 


Nota: Es posible realizar el cálculo de los promedios en una única operación sobre un dataframe, por lo que soluciones que realizan el cálculo del promedio de cada métrica por separado serán consideradas como menos eficientes.



In [2]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [74]:
size_obs = 210
events = pd.DataFrame({
    'uid': rd.choices(range(1,10), k=size_obs),
    'sessionid': range(1,size_obs+1),
    'event': rd.sample(['ecommerce.view-product', 'ecommerce.checkout', 'ecommerce.conversion', 'ecommerce.other'],counts=[int(size_obs/4)+4]*4,k=size_obs),
    'value': rd.choices(range(1,12),k=size_obs),
    'datetime':pd.date_range(start='2021-03-01',freq='S',periods=size_obs) 
})
events

,uid,sessionid,event,value,datetime
0,1,1,ecommerce.view-product,1,2021-03-01 00:00:00
1,5,2,ecommerce.other,7,2021-03-01 00:00:01
2,9,3,ecommerce.view-product,2,2021-03-01 00:00:02
3,2,4,ecommerce.conversion,8,2021-03-01 00:00:03
4,3,5,ecommerce.checkout,11,2021-03-01 00:00:04
...,...,...,...,...,...
205,4,206,ecommerce.checkout,8,2021-03-01 00:03:25
206,3,207,ecommerce.other,10,2021-03-01 00:03:26
207,2,208,ecommerce.conversion,5,2021-03-01 00:03:27
208,7,209,ecommerce.other,5,2021-03-01 00:03:28


## Item a)

In [75]:
events['cant_sesiones'] = events.groupby('uid')['sessionid'].transform('count')
events

,uid,sessionid,event,value,datetime,cant_sesiones
0,1,1,ecommerce.view-product,1,2021-03-01 00:00:00,19
1,5,2,ecommerce.other,7,2021-03-01 00:00:01,25
2,9,3,ecommerce.view-product,2,2021-03-01 00:00:02,22
3,2,4,ecommerce.conversion,8,2021-03-01 00:00:03,30
4,3,5,ecommerce.checkout,11,2021-03-01 00:00:04,22
...,...,...,...,...,...,...
205,4,206,ecommerce.checkout,8,2021-03-01 00:03:25,18
206,3,207,ecommerce.other,10,2021-03-01 00:03:26,22
207,2,208,ecommerce.conversion,5,2021-03-01 00:03:27,30
208,7,209,ecommerce.other,5,2021-03-01 00:03:28,27


In [76]:
promedio = events[['uid','cant_sesiones']].drop_duplicates()['cant_sesiones'].mean()
promedio

23.333333333333332

In [77]:
events.loc[events['cant_sesiones']>promedio,'uid'].drop_duplicates()

1     5
3     2
6     8
17    7
Name: uid, dtype: int64

## Parte b)

In [84]:
events_more_25_ses = events.loc[events['cant_sesiones']>25,:]
mask_categories = (events_more_25_ses['event'] == 'ecommerce.view-product') | (events_more_25_ses['event'] == 'ecommerce.conversion') | (events_more_25_ses['event'] == 'ecommerce.checkout')
resultado = events_more_25_ses.pivot_table(index='uid',columns='event',values='value',aggfunc='mean')
resultado

event,ecommerce.checkout,ecommerce.conversion,ecommerce.other,ecommerce.view-product
uid,,,,
2,5.20,6.43,6.70,5.38
7,7.00,6.10,5.14,3.67
